#### How to use Boosting

`https://keeryang.github.io/papers/Profits_2021Aug28.pdf`

In [85]:
import optuna
from lightgbm import plot_importance, LGBMRegressor
import getFamaFrenchFactors as gff
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [86]:
stocks_df = pd.read_csv('./sp500_stocks.csv')
df = stocks_df.pivot(
    index='Date', columns='Symbol', values='Adj Close')

df = df.reset_index()
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

df.head()

Symbol,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,MTB,MTCH,MTD,MU,PODD,SCHW,STZ,T,TECH,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.122227,4.496877,6.470741,NaN,NaN,18.952162,7.994444,32.212460,37.090000,23.694084,...,44.687515,5.574132,105.639999,10.656090,15.03,16.512775,14.270171,8.236569,9.854395,43.185623
2010-01-05,19.903643,5.005957,6.481929,NaN,NaN,18.799042,7.967778,32.411549,37.700001,23.656675,...,45.487770,5.696431,103.000000,10.970370,15.00,16.462698,14.093122,8.196222,9.769739,43.354244
2010-01-06,19.832930,4.798554,6.378825,NaN,NaN,18.903446,7.933333,32.756096,37.619999,23.611784,...,47.008266,5.650230,102.209999,11.019480,14.67,16.162149,14.279024,8.115704,9.743912,43.728970
2010-01-07,19.807215,4.939964,6.367033,NaN,NaN,19.060045,7.886667,32.725471,36.889999,23.424749,...,48.762154,5.677408,100.440002,10.646270,14.80,16.103729,14.137385,8.024584,9.766868,43.591564
2010-01-08,19.800785,4.845691,6.409364,NaN,NaN,19.157482,7.871111,32.595306,36.689999,23.559416,...,49.242302,5.712738,102.139999,10.901623,14.06,15.995196,13.862957,7.965796,9.748217,43.416687


In [88]:
tickers = ['AAPL']

In [89]:
ff5 = pd.DataFrame(gff.famaFrench5Factor(frequency='m'))
ff5.head()

,date_ff_factors,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-31,-0.0039,-0.0041,-0.0097,0.0068,-0.0118,0.0027
1,1963-08-31,0.0507,-0.0080,0.0180,0.0036,-0.0035,0.0025
2,1963-09-30,-0.0157,-0.0052,0.0013,-0.0071,0.0029,0.0027
3,1963-10-31,0.0253,-0.0139,-0.0010,0.0280,-0.0201,0.0029
4,1963-11-30,-0.0085,-0.0088,0.0175,-0.0051,0.0224,0.0027


In [90]:
fff = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

N_lags = 10

for f in fff:
  for i in range(1, N_lags):
    ff5[f'{f}_{i}L'] = ff5[f].shift(-i)


In [ ]:
mon = pd.DataFrame(df[tickers[0]]).resample('ME').last()
mon_rets = mon.pct_change().dropna()

factors = ff5.rename(columns={'date_ff_factors': 'Date'})
factors_0 = pd.merge(mon_rets, factors, on='Date', how='left')
factors_0 = factors_0.dropna()

Y = (factors_0[tickers[0]] - factors_0['RF'])
X = factors_0.drop(
    columns=['RF', tickers[0]]).set_index('Date')

In [ ]:
top15features_stable = ['RMW_5L',
                        'RMW_2L',
                        'SMB_6L',
                        'CMA_9L',
                        'SMB_2L',
                        'HML_5L',
                        'Mkt-RF_7L',
                        'CMA_4L',
                        'CMA_6L',
                        'RMW_6L',
                        'CMA_8L',
                        'Mkt-RF_2L',
                        'CMA',
                        'RMW',
                        'Mkt-RF']

In [100]:

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, shuffle=False)

## Обучение LSTM

#### How to use LSTM for 5-factor Fama French model

`https://downloads.hindawi.com/journals/ddns/2022/3936122.pdf?_gl=1*8t8631*_ga*MTYwMjYwNDYwNy4xNzE1MzYyMzYy*_ga_NF5QFMJT5V*MTcxNTM2MjM2Mi4xLjEuMTcxNTM2MjQyNC42MC4wLjA.&_ga=2.120926721.845710784.1715362363-1602604607.1715362362`

In [116]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [304]:
import torch.nn as nn
import numpy as np

# Определение размерности входных данных
input_size = 50  # Размерность входного слоя (например, количество признаков)
hidden_size = 10  # Размерность скрытого слоя
num_layers = 10  # Количество слоев LSTM
# Размерность выходного слоя (например, количество прогнозируемых значений)
output_size = 1

# Определение модели LSTM


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)  # Инициализация скрытого состояния
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)  # Инициализация ячеек состояния
        out, _ = self.lstm(x, (h0, c0))  # Прямое распространение через LSTM
        # Применение полносвязного слоя к последнему тензору времени
        out = self.fc(out[:, -1, :])
        return out

In [305]:

# Создание экземпляра модели
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [306]:
# train tensors
X_train_tensor = torch.tensor(
    X_train.loc[:, top15features_stable].values, dtype=torch.float32)
X_train_tensor = X_train_tensor.unsqueeze(1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)


# test tensors
X_test_tensor = torch.tensor(
    X_test.loc[:, top15features_stable].values, dtype=torch.float32)
X_test_tensor = X_test_tensor.unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

model.to(device)
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [307]:
from tqdm import tqdm

# Обучение модели
for epoch in tqdm(range(1000)):  # Пример цикла обучения
    optimizer.zero_grad()  # Очистка градиентов
    outputs = model(X_train_tensor)  # Прогноз модели
    loss = criterion(outputs, y_train_tensor)  # Вычисление потерь
    loss.backward()  # Обратное распространение ошибки
    optimizer.step()  # Обновление весов
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  2%|▏         | 18/1000 [00:00<00:11, 84.44it/s]

Epoch 1, Loss: 0.06641164422035217
Epoch 2, Loss: 0.06497786194086075
Epoch 3, Loss: 0.06356171518564224
Epoch 4, Loss: 0.06216353178024292
Epoch 5, Loss: 0.06078357994556427
Epoch 6, Loss: 0.0594220869243145
Epoch 7, Loss: 0.058079272508621216
Epoch 8, Loss: 0.05675531551241875
Epoch 9, Loss: 0.05545038729906082
Epoch 10, Loss: 0.054164640605449677
Epoch 11, Loss: 0.05289816856384277
Epoch 12, Loss: 0.05165104568004608
Epoch 13, Loss: 0.050423361361026764
Epoch 14, Loss: 0.04921513795852661
Epoch 15, Loss: 0.04802638292312622
Epoch 16, Loss: 0.0468570776283741
Epoch 17, Loss: 0.04570721834897995
Epoch 18, Loss: 0.0445767305791378


  4%|▎         | 36/1000 [00:00<00:11, 83.85it/s]

Epoch 19, Loss: 0.043465543538331985
Epoch 20, Loss: 0.04237357899546623
Epoch 21, Loss: 0.04130074009299278
Epoch 22, Loss: 0.040246929973363876
Epoch 23, Loss: 0.03921203687787056
Epoch 24, Loss: 0.03819593787193298
Epoch 25, Loss: 0.037198495119810104
Epoch 26, Loss: 0.03621961176395416
Epoch 27, Loss: 0.03525914251804352
Epoch 28, Loss: 0.03431697562336922
Epoch 29, Loss: 0.03339296579360962
Epoch 30, Loss: 0.03248700872063637
Epoch 31, Loss: 0.03159896284341812
Epoch 32, Loss: 0.03072870336472988
Epoch 33, Loss: 0.02987612783908844
Epoch 34, Loss: 0.029041092842817307
Epoch 35, Loss: 0.028223492205142975
Epoch 36, Loss: 0.027423208579421043


  4%|▍         | 45/1000 [00:00<00:12, 77.70it/s]

...

Epoch 962, Loss: 0.006180991884320974
Epoch 963, Loss: 0.006180991884320974
Epoch 964, Loss: 0.006180991884320974
Epoch 965, Loss: 0.006180991884320974
Epoch 966, Loss: 0.006180991884320974
Epoch 967, Loss: 0.006180991884320974
Epoch 968, Loss: 0.006180991884320974
Epoch 969, Loss: 0.006180991884320974
Epoch 970, Loss: 0.006180991884320974
Epoch 971, Loss: 0.006180991884320974
Epoch 972, Loss: 0.006180991884320974
Epoch 973, Loss: 0.006180991884320974
Epoch 974, Loss: 0.006180991884320974
Epoch 975, Loss: 0.006180991884320974
Epoch 976, Loss: 0.006180991884320974
Epoch 977, Loss: 0.006180991884320974
Epoch 978, Loss: 0.006180991884320974
Epoch 979, Loss: 0.006180991884320974


 99%|█████████▉| 988/1000 [00:14<00:00, 80.54it/s]

Epoch 980, Loss: 0.006180991884320974
Epoch 981, Loss: 0.006180991884320974
Epoch 982, Loss: 0.006180991884320974
Epoch 983, Loss: 0.006180991884320974
Epoch 984, Loss: 0.006180991884320974
Epoch 985, Loss: 0.006180991884320974
Epoch 986, Loss: 0.006180991884320974
Epoch 987, Loss: 0.006180991884320974
Epoch 988, Loss: 0.006180991884320974
Epoch 989, Loss: 0.006180991884320974
Epoch 990, Loss: 0.006180991884320974
Epoch 991, Loss: 0.006180991884320974
Epoch 992, Loss: 0.006180991884320974
Epoch 993, Loss: 0.006180991884320974
Epoch 994, Loss: 0.006180991884320974
Epoch 995, Loss: 0.006180991884320974
Epoch 996, Loss: 0.006180991884320974


100%|██████████| 1000/1000 [00:14<00:00, 68.89it/s]

Epoch 997, Loss: 0.006180991884320974
Epoch 998, Loss: 0.006180991884320974
Epoch 999, Loss: 0.006180991884320974
Epoch 1000, Loss: 0.006180991884320974


Протестируем модель:

In [308]:
from sklearn.metrics import r2_score


# Выполнение предсказаний на тестовых данных
with torch.no_grad():
    model.eval()  # Установка модели в режим оценки
    y_pred = model(X_test_tensor)  # Предсказания модели
    y_pred = y_pred.cpu().numpy()  # Конвертация предсказаний в NumPy массив

# Вычисление R^2
r2 = r2_score(y_test.to_numpy(), y_pred.reshape(1, -1)[0])
print(f'R^2 score: {r2}')


R^2 score: -0.009415976747705113
